# LLMs

In [34]:
from langchain_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

AZURE_DEPLOYMENT_GPT41 = os.getenv("AZURE_DEPLOYMENT_GPT41")
AZURE_DEPLOYMENT_GPT41_NANO = os.getenv("AZURE_DEPLOYMENT_GPT41_NANO")

gpt41_nano = AzureChatOpenAI(
    azure_deployment=AZURE_DEPLOYMENT_GPT41_NANO,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    temperature=0.0
)

In [35]:
from dotenv import load_dotenv
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq

groqllm = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=GROQ_API_KEY,
    temperature=0
)

# Ontology Rules

In [36]:
from rdflib import Graph, RDF, OWL

def extract_local_names(ttl_file_path):
    """
    Extract ontology elements (local names without full URIs).
    Returns:
        - class_names: ontology classes (entities)
        - object_property_names: ontology object properties (relationships)
        - datatype_property_names: ontology datatype properties (attributes)
    """
    g = Graph()
    g.parse(ttl_file_path, format="turtle")
    
    def get_local_name(uri):
        uri_str = str(uri)
        if '#' in uri_str: 
            return uri_str.split('#')[-1]
        return uri_str.split('/')[-1]
    
    # Classes
    class_uris = [s for s, _, _ in g.triples((None, RDF.type, OWL.Class))]
    class_names = [get_local_name(uri) for uri in class_uris]
    
    # Object properties
    object_prop_uris = [s for s, _, _ in g.triples((None, RDF.type, OWL.ObjectProperty))]
    object_property_names = [get_local_name(uri) for uri in object_prop_uris]
    
    # Datatype properties
    data_prop_uris = [s for s, _, _ in g.triples((None, RDF.type, OWL.DatatypeProperty))]
    datatype_property_names = [get_local_name(uri) for uri in data_prop_uris]
    
    return (
        sorted(class_names),
        sorted(object_property_names),
        sorted(datatype_property_names),
    )

# Usage
classes, relations, attributes = extract_local_names('output/ontologies/RDB/rigor_ontology.ttl')

print(f"Found {len(classes)} classes, {len(relations)} object properties, {len(attributes)} datatype properties")
print("\nClass names:", classes)
print("\nObject properties (relations):", relations)
print("\nDatatype properties (attributes):", attributes)

Found 43 classes, 55 object properties, 126 datatype properties

Class names: ['Company', 'Criterion', 'GeographicCriterion', 'Grant', 'GrantPayment', 'GrantShare', 'GranterApplication', 'GranterApplicationFile', 'GranterCompany', 'GranterCompanyFile', 'GranterGeneralOpportunityFile', 'GranterOpportunity', 'GranterOpportunityFile', 'GranterPartner', 'OrganisationalCriterion', 'RelatedTable', 'agency_identifier', 'career_purpose_criterion', 'criterion', 'criterion_description', 'funding_scheme_criteria', 'granterCompany', 'granterCompanyMemory', 'granterConsortium', 'granterPartnerType', 'granter_applicationfile', 'granter_applications', 'granter_company', 'granter_companyfile', 'granter_companymemory', 'granter_consortium', 'granter_consortium_part_type', 'granter_eligibilitycriteria', 'granter_generalopportunityfile', 'granter_generalopportunityfile_opportunities', 'granter_matchcheck', 'granter_matchgroup', 'granter_opportunity', 'granter_opportunityfiles', 'granter_profile', 'grante

# Input text

In [37]:
from langchain_core.documents import Document
import os

input_txt_folder = "data/texts"

# Import opportunity file 
for filename in os.listdir(input_txt_folder):
    if filename.endswith('opportunity_example.txt'):
        file_path = os.path.join(input_txt_folder, filename)
        print(f"Processing {filename}...")

        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                print(type(content))
        except Exception as e:
            print(f"Error processing {filename}: {e}")

document = Document(page_content=content)
print(document)

Processing opportunity_example.txt...
<class 'str'>
page_content='Os Fundos Europeus mais próximos de si.  
           
 
          1/23 
Aviso para apresentação de  candidaturas  
Código do aviso   MAR2030 -2023 -22 
Data de publicação  14/11 /2023  
Natureza do aviso   Concurso  
Âmbito de atuação:  Operações  
Aprovado pelo SRMAR a 25/07/2023  
 
 
Designação do  aviso  
Transformação  de Produtos da Pesca e da Aquicultura no Domínio dos Investimentos Produtivos -  Região 
Autónoma da Madeira  
 
Apoio para  
Promover a comercialização, a qualidade, o valor acrescentado dos produtos da pesca e da aquicultura, assim 
como a transformação destes produtos . 
Ações abrangidas por este aviso  
São abrangidas pelo presente aviso as ações, promovidas por empresas, previstas no artigo 50.º da Portaria n.º 
559/2023, de 25 de julho , relativas a : 
a) Investimentos produtivos bem como investimentos que promovam a descarbonização, o uso de energias 
renováveis e a eficiência energética, a eco

## Chunking it

In [38]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_document(document, chunk_size=3000, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return text_splitter.split_documents([document])

chunks = chunk_document(document)
print(chunks[0].page_content)

Os Fundos Europeus mais próximos de si.  
           
 
          1/23 
Aviso para apresentação de  candidaturas  
Código do aviso   MAR2030 -2023 -22 
Data de publicação  14/11 /2023  
Natureza do aviso   Concurso  
Âmbito de atuação:  Operações  
Aprovado pelo SRMAR a 25/07/2023  
 
 
Designação do  aviso  
Transformação  de Produtos da Pesca e da Aquicultura no Domínio dos Investimentos Produtivos -  Região 
Autónoma da Madeira  
 
Apoio para  
Promover a comercialização, a qualidade, o valor acrescentado dos produtos da pesca e da aquicultura, assim 
como a transformação destes produtos . 
Ações abrangidas por este aviso  
São abrangidas pelo presente aviso as ações, promovidas por empresas, previstas no artigo 50.º da Portaria n.º 
559/2023, de 25 de julho , relativas a : 
a) Investimentos produtivos bem como investimentos que promovam a descarbonização, o uso de energias 
renováveis e a eficiência energética, a economia circular, a digitalização e a internacionalização,  
b) Inve

# Prompt

In [39]:
test_sentence = chunks[0].page_content
print(test_sentence)

Os Fundos Europeus mais próximos de si.  
           
 
          1/23 
Aviso para apresentação de  candidaturas  
Código do aviso   MAR2030 -2023 -22 
Data de publicação  14/11 /2023  
Natureza do aviso   Concurso  
Âmbito de atuação:  Operações  
Aprovado pelo SRMAR a 25/07/2023  
 
 
Designação do  aviso  
Transformação  de Produtos da Pesca e da Aquicultura no Domínio dos Investimentos Produtivos -  Região 
Autónoma da Madeira  
 
Apoio para  
Promover a comercialização, a qualidade, o valor acrescentado dos produtos da pesca e da aquicultura, assim 
como a transformação destes produtos . 
Ações abrangidas por este aviso  
São abrangidas pelo presente aviso as ações, promovidas por empresas, previstas no artigo 50.º da Portaria n.º 
559/2023, de 25 de julho , relativas a : 
a) Investimentos produtivos bem como investimentos que promovam a descarbonização, o uso de energias 
renováveis e a eficiência energética, a economia circular, a digitalização e a internacionalização,  
b) Inve

In [40]:
examples = [
    {'sentence': '<id> The technology company submitted their grant application for AI research projects.', 
    'answer': '(technology company, appliedFor, grant application)'},

    {'sentence': '<id> The granter opportunity requires applicants to meet the organisational criterion of having at least 50 employees.', 
    'answer': '(granter opportunity, requires, organisational criterion)\n(applicants, hasEligibility, organisational criterion)'},

    {'sentence': '<id> The healthcare company applied for a grant that offers payments of up to $500,000.', 
    'answer': '(healthcare company, appliesFor, grant)\n(grant, offers, grant payment)'},

    {'sentence': '<id> The environmental grant has a deadline of September 30 for all project submissions.', 
    'answer': '(environmental grant, hasDeadline, September 30)\n(environmental grant, requires, project submissions)'},

    {'sentence': '<id> The granter application file was evaluated by an AI review state before final approval.', 
    'answer': '(AI review state, hasEvaluates, granter application file)\n(granter application file, aiReviewState, AI review state)'},
]


In [41]:
# - Allowed Attributes (datatype properties): {attributes_str}

def generate_prompt(concepts, relations, attributes, examples, test_sentence):
    """
    Build an ontology-guided extraction prompt for LLMs.
    
    Args:
        concepts (list[str]): Ontology classes (entities).
        relations (list[str]): Ontology object properties (relationships).
        attributes (list[str]): Ontology datatype properties (attributes).
        examples (list[dict]): Few-shot examples with keys {"sentence": str, "answer": str}.
        test_sentence (str): The new sentence to extract triples from.
    
    Returns:
        str: The formatted LLM prompt.
    """
    # Format ontology context
    concepts_str = ", ".join(concepts)
    relations_str = ", ".join(relations)
    attributes_str = ", ".join(attributes)

    # Format examples
    examples_str = "\n\n".join([
        f"Example sentence: {ex['sentence']}\nExample answer: {ex['answer']}"
        for ex in examples
    ])
    
    # Build complete prompt
    return f"""Given the following ontology and sentence, please extract the triples from the sentence according to the relations in the ontology.
In the output, only include the triples in the given output format

Context:
- Ontology Entities (classes): {concepts_str}
- Ontology Relations (object properties): {relations_str}

For each input sentence, output triples in the following formats:
- Relationship triple: (subject_entity, relationship, object_entity)
- Attribute triple: (entity, attribute, literal_value)

{examples_str}

Given these examples, give the correct answer for the following sentence:
Test sentence:\n{test_sentence}\n
Test answer:"""

print(generate_prompt(classes, relations, attributes, examples, test_sentence))

Given the following ontology and sentence, please extract the triples from the sentence according to the relations in the ontology.
In the output, only include the triples in the given output format

Context:
- Ontology Entities (classes): Company, Criterion, GeographicCriterion, Grant, GrantPayment, GrantShare, GranterApplication, GranterApplicationFile, GranterCompany, GranterCompanyFile, GranterGeneralOpportunityFile, GranterOpportunity, GranterOpportunityFile, GranterPartner, OrganisationalCriterion, RelatedTable, agency_identifier, career_purpose_criterion, criterion, criterion_description, funding_scheme_criteria, granterCompany, granterCompanyMemory, granterConsortium, granterPartnerType, granter_applicationfile, granter_applications, granter_company, granter_companyfile, granter_companymemory, granter_consortium, granter_consortium_part_type, granter_eligibilitycriteria, granter_generalopportunityfile, granter_generalopportunityfile_opportunities, granter_matchcheck, granter_ma

# Trying without structured output

In [42]:
answer = groqllm.invoke(generate_prompt(classes, relations, attributes, examples, test_sentence))

In [43]:
print(answer.content)

Based on the provided ontology and the given sentence, I will extract the triples according to the relations in the ontology.

Here are the extracted triples:

1. (Os Fundos Europeus, relatesTo, agency_identifier)
2. (Os Fundos Europeus, hasProvenance, 14/11/2023)
3. (Os Fundos Europeus, hasProvenance, 25/07/2023)
4. (Os Fundos Europeus, hasProvenance, 25/07/2023)
5. (Os Fundos Europeus, hasProvenance, MAR2030-2023-22)
6. (Os Fundos Europeus, hasProvenance, Concurso)
7. (Os Fundos Europeus, hasProvenance, Operações)
8. (Os Fundos Europeus, hasProvenance, Transformação de Produtos da Pesca e da Aquicultura no Domínio dos Investimentos Produtivos - Região Autónoma da Madeira)
9. (Os Fundos Europeus, hasProvenance, Promover a comercialização, a qualidade, o valor acrescentado dos produtos da pesca e da aquicultura, assim como a transformação destes produtos)
10. (Os Fundos Europeus, hasProvenance, Investimentos produtivos bem como investimentos que promovam a descarbonização, o uso de ene

# With LLMGraphTransformer

In [44]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
graph_transformer = LLMGraphTransformer(llm = gpt41_nano,
                                        allowed_nodes = classes,
                                        allowed_relationships = relations)

In [45]:
graph_documents = await graph_transformer.aconvert_to_graph_documents(chunks)

In [46]:
graph_documents

[GraphDocument(nodes=[Node(id='Mar2030 -2023 -22', type='Relatedtable', properties={}), Node(id='14/11/2023', type='Relatedtable', properties={}), Node(id='Concurso', type='Criterion', properties={}), Node(id='Operações', type='Criterion', properties={}), Node(id='25/07/2023', type='Relatedtable', properties={}), Node(id='Transformação De Produtos Da Pesca E Da Aquicultura No Domínio Dos Investimentos Produtivos - Região Autónoma Da Madeira', type='Grant', properties={}), Node(id='Fundos Europeus Mais Próximos De Si', type='Grant', properties={}), Node(id='Portaria N.º 559/2023', type='Relatedtable', properties={}), Node(id='Artigo 50.º Da Portaria N.º 559/2023', type='Relatedtable', properties={}), Node(id='Artigo 23.º Do Dl 20 -A/2023', type='Relatedtable', properties={}), Node(id='Start-Ups', type='Criterion', properties={}), Node(id='Spin-Offs', type='Criterion', properties={}), Node(id='Marcas', type='Criterion', properties={}), Node(id='Patentes', type='Criterion', properties={})

In [47]:
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Mar2030 -2023 -22', type='Relatedtable', properties={}), Node(id='14/11/2023', type='Relatedtable', properties={}), Node(id='Concurso', type='Criterion', properties={}), Node(id='Operações', type='Criterion', properties={}), Node(id='25/07/2023', type='Relatedtable', properties={}), Node(id='Transformação De Produtos Da Pesca E Da Aquicultura No Domínio Dos Investimentos Produtivos - Região Autónoma Da Madeira', type='Grant', properties={}), Node(id='Fundos Europeus Mais Próximos De Si', type='Grant', properties={}), Node(id='Portaria N.º 559/2023', type='Relatedtable', properties={}), Node(id='Artigo 50.º Da Portaria N.º 559/2023', type='Relatedtable', properties={}), Node(id='Artigo 23.º Do Dl 20 -A/2023', type='Relatedtable', properties={}), Node(id='Start-Ups', type='Criterion', properties={}), Node(id='Spin-Offs', type='Criterion', properties={}), Node(id='Marcas', type='Criterion', properties={}), Node(id='Patentes', type='Criterion', properties={}), Node(id='Espé

In [48]:
def merge_graph_documents(graph_documents):
    """Simple function to merge all nodes and relationships from graph documents."""
    all_nodes = []
    all_relationships = []
    
    # Collect everything
    for doc in graph_documents:
        all_nodes.extend(doc.nodes)
        all_relationships.extend(doc.relationships)
    
    # Remove duplicate nodes by ID
    seen_nodes = set()
    unique_nodes = []
    for node in all_nodes:
        if node.id not in seen_nodes:
            seen_nodes.add(node.id)
            unique_nodes.append(node)
    
    # Remove duplicate relationships
    seen_rels = set()
    unique_relationships = []
    for rel in all_relationships:
        rel_key = (rel.source.id, rel.target.id, rel.type)
        if rel_key not in seen_rels:
            seen_rels.add(rel_key)
            unique_relationships.append(rel)
    
    # Create merged document
    merged_doc = type(graph_documents[0])(
        nodes=unique_nodes,
        relationships=unique_relationships,
        source=graph_documents[0].source
    )
    
    print(f"Merged: {len(unique_nodes)} nodes, {len(unique_relationships)} relationships")
    return [merged_doc]

# Use it
graph_documents = merge_graph_documents(graph_documents)
graph_documents

Merged: 285 nodes, 48 relationships


[GraphDocument(nodes=[Node(id='Mar2030 -2023 -22', type='Relatedtable', properties={}), Node(id='14/11/2023', type='Relatedtable', properties={}), Node(id='Concurso', type='Criterion', properties={}), Node(id='Operações', type='Criterion', properties={}), Node(id='25/07/2023', type='Relatedtable', properties={}), Node(id='Transformação De Produtos Da Pesca E Da Aquicultura No Domínio Dos Investimentos Produtivos - Região Autónoma Da Madeira', type='Grant', properties={}), Node(id='Fundos Europeus Mais Próximos De Si', type='Grant', properties={}), Node(id='Portaria N.º 559/2023', type='Relatedtable', properties={}), Node(id='Artigo 50.º Da Portaria N.º 559/2023', type='Relatedtable', properties={}), Node(id='Artigo 23.º Do Dl 20 -A/2023', type='Relatedtable', properties={}), Node(id='Start-Ups', type='Criterion', properties={}), Node(id='Spin-Offs', type='Criterion', properties={}), Node(id='Marcas', type='Criterion', properties={}), Node(id='Patentes', type='Criterion', properties={})

In [49]:
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Mar2030 -2023 -22', type='Relatedtable', properties={}), Node(id='14/11/2023', type='Relatedtable', properties={}), Node(id='Concurso', type='Criterion', properties={}), Node(id='Operações', type='Criterion', properties={}), Node(id='25/07/2023', type='Relatedtable', properties={}), Node(id='Transformação De Produtos Da Pesca E Da Aquicultura No Domínio Dos Investimentos Produtivos - Região Autónoma Da Madeira', type='Grant', properties={}), Node(id='Fundos Europeus Mais Próximos De Si', type='Grant', properties={}), Node(id='Portaria N.º 559/2023', type='Relatedtable', properties={}), Node(id='Artigo 50.º Da Portaria N.º 559/2023', type='Relatedtable', properties={}), Node(id='Artigo 23.º Do Dl 20 -A/2023', type='Relatedtable', properties={}), Node(id='Start-Ups', type='Criterion', properties={}), Node(id='Spin-Offs', type='Criterion', properties={}), Node(id='Marcas', type='Criterion', properties={}), Node(id='Patentes', type='Criterion', properties={}), Node(id='Espé

## Visualize

In [50]:
from pyvis.network import Network

def visualize_graph(graph_documents):

    # Create network
    net = Network(height="1200px", width="100%", directed=True,
                      notebook=False, bgcolor="#222222", font_color="white")
    
    nodes = graph_documents[0].nodes
    relationships = graph_documents[0].relationships

    # Build lookup for valid nodes
    node_dict = {node.id: node for node in nodes}
    
    # Filter out invalid edges and collect valid node IDs
    valid_edges = []
    valid_node_ids = set()
    for rel in relationships:
        if rel.source.id in node_dict and rel.target.id in node_dict:
            valid_edges.append(rel)
            valid_node_ids.update([rel.source.id, rel.target.id])


    # Track which nodes are part of any relationship
    connected_node_ids = set()
    for rel in relationships:
        connected_node_ids.add(rel.source.id)
        connected_node_ids.add(rel.target.id)

    # Add valid nodes
    for node_id in valid_node_ids:
        node = node_dict[node_id]
        try:
            net.add_node(node.id, label=node.id, title=node.type, group=node.type)
        except:
            continue  # skip if error

    # Add valid edges
    for rel in valid_edges:
        try:
            net.add_edge(rel.source.id, rel.target.id, label=rel.type.lower())
        except:
            continue  # skip if error

    # Configure physics
    net.set_options("""
            {
                "physics": {
                    "forceAtlas2Based": {
                        "gravitationalConstant": -100,
                        "centralGravity": 0.01,
                        "springLength": 200,
                        "springConstant": 0.08
                    },
                    "minVelocity": 0.75,
                    "solver": "forceAtlas2Based"
                }
            }
            """)
        
    output_file = "knowledge_graph.html"
    net.save_graph(output_file)
    print(f"Graph saved to {os.path.abspath(output_file)}")

    # Try to open in browser
    try:
        import webbrowser
        webbrowser.open(f"file://{os.path.abspath(output_file)}")
    except:
        print("Could not open browser automatically")
        
# Run the function
visualize_graph(graph_documents)

Graph saved to c:\Users\tiago\Documents\Granter Ai Internship\Implementation\KGs_for_Vertical_AI\knowledge_graph.html


# Saving to csv

In [ ]:
import pandas as pd
import os

def save_graph_to_csv(graph_documents, output_file="knowledge_graph.csv"):
    """
    Save the knowledge graph to CSV format with structure (object, relation, subject).
    
    Args:
        graph_documents: The graph documents from LLMGraphTransformer
        output_file: Output CSV filename
    """
    relationships = graph_documents[0].relationships
    
    # Extract triples in (object, relation, subject) format
    triples = []
    for rel in relationships:
        triple = {
            'object': rel.source.id,      # source node as object
            'relation': rel.type,         # relationship type
            'subject': rel.target.id      # target node as subject
        }
        triples.append(triple)
    
    # Create DataFrame and save to CSV
    df = pd.DataFrame(triples)
    df.to_csv(output_file, index=False)
    
    print(f"Graph saved to CSV: {os.path.abspath(output_file)}")
    print(f"Total triples: {len(triples)}")
    
    # Display first few rows
    print(df.head())
    
    return df

# Save the graph to CSV
df = save_graph_to_csv(graph_documents)

Graph saved to CSV: c:\Users\tiago\Documents\Granter Ai Internship\Implementation\KGs_for_Vertical_AI\knowledge_graph.csv
Total triples: 48

First 5 triples:
                           object        relation            subject
0               Mar2030 -2023 -22  HASAPPLICATION           Concurso
1                      14/11/2023      CREATED_BY         25/07/2023
2               Despesas_Veiculos  RELATEDTOTABLE           Despesas
3        Limite_Despesas_Veiculos  RELATEDTOTABLE  Despesas_Veiculos
4  Despesas_Auditoria_Consultoria  RELATEDTOTABLE           Despesas
